In [1]:
import time
import datetime
from mkndaq.inst.neph import NEPH
import schedule
import threading

cfg = {'ne300': {
                'type': 'NE300',
                'serial_number': '23-0690',
                'serial_id': 0,
                'protocol': 'acoem',
                'socket': {
                    'host': '192.168.0.50',
                    'port': 32783,
                    'timeout': 10,
                },
                'get_data_interval': 3, # this is the logger retrieval interval
                'zero_span_check_interval': 7,
                'zero_check_duration': 3,
                'span_check_duration': 2,
                'staging_zip': True,  
                'verbosity': 2,  # 0: silent, 1: medium, 2: full          
            },
            'reporting_interval': 10,
            'logs': 'mkndaq/logs',
            'data': 'mkndaq/data',
            'staging': {
                'path': 'mkndaq/staging',
            },
}
ne300 = NEPH('ne300', cfg, verbosity=0)
# # Initialize NEPH (name: ne300  S/N: 23-0690)
#   Instrument identified itself as '{'Model': 158, 'Variant': 300, 'Sub-Type': 0, 'Range': 0, 'Build': 158, 'Branch': 300}'.

# Initialize NEPH (name: ne300  S/N: 23-0690)
  - Instrument identified itself as '{'id': 'ACOEM Aurora NE-300 Sub-Type: 0 Range: 0 Build: 158 Branch: 300'}'.
  - Instrument current operation: ambient.
2024-05-24 09:39:41 [ne300] dtm found: 2024-05-24 09:39:40 > dtm after set: 2024-05-24 09:39:40.


In [2]:
def run_threaded(job_func):
    """Set up threading and start job.

    Args:
        job_func ([type]): [description]
    """
    job_thread = threading.Thread(target=job_func)
    job_thread.start()

# align start with a 10' timestamp
while int(time.time()) % 10 > 0:
    time.sleep(0.1)

fetch = 20
schedule.every(fetch).seconds.do(ne300.print_ssp_bssp)
schedule.every(cfg['ne300']['get_data_interval']).minutes.at(':00').do(run_threaded, ne300.get_new_data)
schedule.every(cfg['ne300']['zero_span_check_interval']).minutes.at(':00').do(run_threaded, ne300.do_zero_span_check)

while True:
    schedule.run_pending()
    time.sleep(1)

2024-05-24 09:40:10 [ne300] ssp|bssp (Mm-1) r: -0.2498|-0.6653 g: 0.7726|0.1490 b: 0.6757|0.2156
2024-05-24 09:40:30 [ne300] ssp|bssp (Mm-1) r: -0.2514|-0.6695 g: 0.7803|0.1386 b: 0.6978|0.2113
2024-05-24 09:40:50 [ne300] ssp|bssp (Mm-1) r: -0.2136|-0.6632 g: 0.8241|0.1238 b: 0.7104|0.2129
2024-05-24 09:41:10 [ne300] ssp|bssp (Mm-1) r: -0.2085|-0.6660 g: 0.8367|0.1199 b: 0.7152|0.2146
2024-05-24 09:41:30 [ne300] ssp|bssp (Mm-1) r: -0.2366|-0.6775 g: 0.7987|0.1148 b: 0.7132|0.2128
2024-05-24 09:41:50 [ne300] ssp|bssp (Mm-1) r: -0.2670|-0.6955 g: 0.7712|0.1226 b: 0.7102|0.2127
2024-05-24 09:42:00 .get_new_data (name=ne300, save=True)
2024-05-24 09:42:10 [ne300] ssp|bssp (Mm-1) r: -0.2393|-0.6941 g: 0.8414|0.1078 b: 0.7190|0.2075
2024-05-24 09:42:30 [ne300] ssp|bssp (Mm-1) r: -0.0743|-0.7004 g: 0.8043|0.1291 b: 0.7163|0.2002
2024-05-24 09:42:50 [ne300] ssp|bssp (Mm-1) r: 0.0708|-0.6911 g: 0.9130|0.1271 b: 0.7253|0.2006
2024-05-24 09:43:10 [ne300] ssp|bssp (Mm-1) r: 0.1154|-0.7075 g: 1.049

KeyboardInterrupt: 

In [ ]:
res = ne300.get_id()
print(res)

In [ ]:
# mimick VI099 response
ne300.get_current_data(strict=True, verbosity=0)

In [ ]:
ne300.get_current_data(verbosity=0)

In [ ]:
ne300.get_values([1635000, 1525000], verbosity=2)

In [ ]:
# retrieve VI099 command parameters
ne300.get_values(parameters=[1, 1635000,1525000,1450000,1635090,1525090,1450090,5001,5004,5003,5002,4036,4035])

In [ ]:
ne300.get_current_data(add_params=[1001, 1002, 1003, 1004, 1005, 2001, 2002])

In [ ]:
# not implemented
ne300.get_new_data()

In [ ]:
# not implemented
ne300.get_all_data()

In [ ]:
ne300.get_instr_type()

In [ ]:
ne300.get_datetime()

In [ ]:
ne300.get_logging_config()

In [ ]:
# cycle through operating states
# 0: ambient, 1: zero, 2: span
for i in [1, 2, 0]:
    print(f"current: {ne300.get_current_operation()} > set: {i} ...")
    print(f"done: {ne300.set_current_operation(state=i)} okay")
    time.sleep(3)

In [ ]:
from datetime import datetime, timezone, timedelta
end = datetime.now(timezone.utc)
start = end - timedelta(minutes=10)
print(f"start: {start}\nend  : {end}")
data = ne300.get_logged_data(start=start, end=end, verbosity=1)
# for i in range(len(data)):
    # print(data[i])

In [ ]:
# repeat packet
message = ne300._acoem_construct_message(command=7, payload=bytes([0,0,0,1]))
response = ne300.tcpip_comm(message=message)
print(response)
display(ne300._acoem_decode_logged_data(response=response))

In [ ]:
# next packet
message = ne300._acoem_construct_message(command=7, payload=bytes([0,0,0,0]))
response = ne300.tcpip_comm(message=message)
response